In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy.interpolate import interp1d

def CalVoltage(C1, C2, R0, R1, R2, gamma1, M0, M, i, ir1=0, ir2=0, z=100, h=0, s=0):
    Ts = 0.0001 
    Q = 10 * 3600  
    n = 1  

    z = z - Ts * n * i / Q

    ir1 = np.exp(-Ts / (R1 * C1)) * ir1 + (1 - np.exp(-Ts / (R1 * C1))) * i
    ir2 = np.exp(-Ts / (R2 * C2)) * ir2 + (1 - np.exp(-Ts / (R2 * C2))) * i

    u = np.exp(-abs(n * i * gamma1 * Ts / Q))
    h = u * h - (1 - u) * (1.0 if i > 0 else 0.0)
    s = 1 if i > 0 else s
    vh = M0 * s + M * h

    v = OCV(z) + vh - R1 * ir1 - R2 * ir2 - R0 * i
    return round(v, 4), ir1, ir2, z, h, s

def OCV(x_val):
    y = np.array([2.5, 2.5999, 2.757, 3.0026, 3.1401, 3.2088, 3.2383, 
                  3.2726, 3.2972, 3.3119, 3.3119, 3.3365, 3.3709, 3.4887, 3.5])
    x = np.array([0, 0.18474, 0.71411, 3.5374, 7.243, 12.36, 20.124, 
                  32.3, 44.828, 60.0004, 70.591, 84.708, 97.413, 99.707, 100])
    ocv_interp = interp1d(x, y, kind='linear', fill_value="extrapolate")
    return float(ocv_interp(x_val))

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

data_path = 'modified10.xlsx'
data = pd.read_excel(data_path)
data_input = data.iloc[:, :-1].values  
current_sense = data['Current_sense'].values
terminal_voltage = data['Terminal_voltage'].values

num_outputs = 8
input_dim = data_input.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu', input_shape=(input_dim,)),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),                             
    tf.keras.layers.Dense(num_outputs)  
])

model.compile(optimizer='adam', loss=rmse)

outputs = model.predict(data_input)
a = ['C1', 'C2', 'R0', 'R1', 'R2', 'gamma1', 'M0', 'M']


for i in range(25):
    for j in range(8):
        print(a[j], " = ", outputs[i][j])

    voltage = CalVoltage(*outputs[i], data['Current_sense'][i])[0]

    loss = rmse(tf.constant([voltage], dtype=tf.float32), tf.constant([data['Terminal_voltage'][i]], dtype=tf.float32))
    
    print("Voltage at these parameters = ", voltage, " at current = ", data['Current_sense'][i])
    print("LOSS = ", loss.numpy())
    print('\n')



d:\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3126/3126 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
C1  =  -0.30397102
C2  =  0.142459
R0  =  -0.024617866
R1  =  0.15684614
R2  =  -0.07903986
gamma1  =  -0.32713878
M0  =  -0.33706337
M  =  -0.5359121
Voltage at these parameters =  3.163  at current =  0.0016
LOSS =  0.339


C1  =  -0.30401152
C2  =  0.14288355
R0  =  -0.024822488
R1  =  0.15625152
R2  =  -0.07931405
gamma1  =  -0.32697707
M0  =  -0.33708185
M  =  -0.53524745
Voltage at these parameters =  3.1631  at current =  0.0054
LOSS =  0.3389001


C1  =  -0.30402017
C2  =  0.14322038
R0  =  -0.025078759
R1  =  0.15558234
R2  =  -0.07969773
gamma1  =  -0.3266812
M0  =  -0.3370596
M  =  -0.53455156
Voltage at these parameters =  3.1632  at current =  0.0085
LOSS =  0.33870006


C1  =  -0.30405873
C2  =  0.14365584
R0  =  -0.02528888
R1  =  0.15496346
R2  =  -0.07998948
gamma1  =  -0.326492
M0  =  -0.33709472
M  =  -0.5338543
Voltage at these parameters =  3.1632  at current =  0.0125
LOSS =  0.33870006


C1  =  -0.30408165
C2  =  0.14399

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy.interpolate import interp1d

def CalVoltage(C1, C2, R0, R1, R2, gamma1, M0, M, i, ir1=0, ir2=0, z=100, h=0, s=0):
    Ts = 0.0001 
    Q = 10 * 3600  
    n = 1  

    z = z - Ts * n * i / Q
    ir1 = np.exp(-Ts / (R1 * C1)) * ir1 + (1 - np.exp(-Ts / (R1 * C1))) * i
    ir2 = np.exp(-Ts / (R2 * C2)) * ir2 + (1 - np.exp(-Ts / (R2 * C2))) * i
    u = np.exp(-abs(n * i * gamma1 * Ts / Q))
    h = u * h - (1 - u) * (1.0 if i > 0 else 0.0)
    s = 1 if i > 0 else s
    vh = M0 * s + M * h
    v = OCV(z) + vh - R1 * ir1 - R2 * ir2 - R0 * i
    return round(v, 4), ir1, ir2, z, h, s

def OCV(x_val):
    y = np.array([2.5, 2.5999, 2.757, 3.0026, 3.1401, 3.2088, 3.2383, 
                  3.2726, 3.2972, 3.3119, 3.3119, 3.3365, 3.3709, 3.4887, 3.5])
    x = np.array([0, 0.18474, 0.71411, 3.5374, 7.243, 12.36, 20.124, 
                  32.3, 44.828, 60.0004, 70.591, 84.708, 97.413, 99.707, 100])
    ocv_interp = interp1d(x, y, kind='linear', fill_value="extrapolate")
    return float(ocv_interp(x_val))

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

data_path = 'modified10.xlsx'
data = pd.read_excel(data_path)
data_input = data.iloc[:, :-1].values  
current_sense = data['Current_sense'].values
terminal_voltage = data['Terminal_voltage'].values

num_outputs = 8
input_dim = data_input.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu', input_shape=(input_dim,)),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),                             
    tf.keras.layers.Dense(num_outputs)  
])

model.compile(optimizer='adam', loss=rmse)

outputs = model.predict(data_input)

predicted_voltages = []

parameter_names = ['C1', 'C2', 'R0', 'R1', 'R2', 'gamma1', 'M0', 'M']

for i in range(25):
    print(f"Sample {i+1} Predicted Parameters:")
    for j in range(8):
        print(f"{parameter_names[j]} = {outputs[i][j]:.4f}")

    voltage, *_ = CalVoltage(*outputs[i], current_sense[i])
    predicted_voltages.append(voltage)

    print(f"Predicted Voltage = {voltage:.4f} V at Current = {current_sense[i]} A")
    print(f"Actual Voltage = {terminal_voltage[i]:.4f} V")
    
    loss = rmse(tf.constant([voltage], dtype=tf.float32), tf.constant([terminal_voltage[i]], dtype=tf.float32))
    print(f"LOSS for Sample {i+1} = {loss.numpy():.4f}\n")

predicted_voltages_tf = tf.constant(predicted_voltages, dtype=tf.float32)
actual_voltages_tf = tf.constant(terminal_voltage[:25], dtype=tf.float32)

final_loss = rmse(actual_voltages_tf, predicted_voltages_tf)
print(f"Final RMSE Loss over all predicted voltages: {final_loss.numpy():.4f}")


d:\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3126/3126 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
Sample 1 Predicted Parameters:
C1 = 0.2703
C2 = 0.4258
R0 = 0.5514
R1 = 0.8659
R2 = -0.2491
gamma1 = -0.6238
M0 = 0.4601
M = -0.6469
Predicted Voltage = 3.9592 V at Current = 0.0016 A
Actual Voltage = 3.5020 V
LOSS for Sample 1 = 0.4572

Sample 2 Predicted Parameters:
C1 = 0.2704
C2 = 0.4254
R0 = 0.5513
R1 = 0.8655
R2 = -0.2496
gamma1 = -0.6229
M0 = 0.4598
M = -0.6465
Predicted Voltage = 3.9568 V at Current = 0.0054 A
Actual Voltage = 3.5020 V
LOSS for Sample 2 = 0.4548

Sample 3 Predicted Parameters:
C1 = 0.2704
C2 = 0.4249
R0 = 0.5512
R1 = 0.8651
R2 = -0.2502
gamma1 = -0.6219
M0 = 0.4597
M = -0.6460
Predicted Voltage = 3.9550 V at Current = 0.0085 A
Actual Voltage = 3.5019 V
LOSS for Sample 3 = 0.4531

Sample 4 Predicted Parameters:
C1 = 0.2705
C2 = 0.4245
R0 = 0.5511
R1 = 0.8646
R2 = -0.2507
gamma1 = -0.6210
M0 = 0.4594
M = -0.6455
Predicted Voltage = 3.9525 V at Current = 0.0125 A
Actual Voltage = 3.5019 V
LOSS for Sample 4 = 0.4506

Sampl

In [16]:
for i, layer in enumerate(model.layers):
    weights, biases = layer.get_weights()
    print(f"Layer {i+1} Weights:\n{weights}")
    print(f"Layer {i+1} Biases:\n{biases}\n")


Layer 1 Weights:
[[ 0.03355679 -0.07036018 -0.38751543  0.08221871 -0.06879041  0.09413874
  -0.1665102   0.09297633  0.32271034 -0.0853678  -0.39017597  0.41777945
  -0.14565012  0.3727213  -0.06863946]
 [ 0.32645142  0.44803387 -0.3886869   0.15900654 -0.3834771   0.42069113
   0.23463887  0.02020147 -0.3787266   0.05463398  0.20542425  0.1009962
   0.2240001   0.06145841 -0.19029182]
 [-0.4052351   0.1744188   0.2275551  -0.3930485  -0.31305403 -0.3205911
   0.20383215 -0.09812635  0.430398   -0.2997858   0.05669558  0.22816253
  -0.44101322  0.12259811 -0.03007066]
 [ 0.38442665 -0.18529806 -0.04147917 -0.16387093 -0.19442353 -0.02107555
   0.4425434   0.22137475  0.25609475  0.19612259 -0.33817756 -0.02866596
   0.22926193  0.31877232 -0.29062963]
 [ 0.01817882 -0.01940915 -0.39244813  0.09386992 -0.13856703 -0.01203358
   0.05317539  0.20212215 -0.07344797 -0.21730363 -0.27684766  0.04694533
  -0.25681722 -0.21754222 -0.2985021 ]
 [-0.43763337  0.3909493  -0.20547773 -0.42911065 